# Assignment 5
## Due May 23 at 12:00

Please note: 

- Read the instructions in the exercise PDF and in this notebook carefully.
- Add your solutions *only* at `YOUR CODE HERE`/`YOUR ANSWER HERE` and remove the corresponding `raise NotImplementedError()`.
- Do not chance the provided code and text, if not stated.
- Do not *add* or *delete* cells.
- Do not `import` additional functionality. 
- Before submitting: Please make sure, that your notebook can be executed from top to bottom `Menu -> Kernel -> Restart & Run all`. 

## Exercise 1
(a) First, the optimization problem has to be reformulated in standard form:

minimize $f(x,y) = -x-y$

s.t. $g(x,y) = x^2 + 2 y^2 -5 = 0$

The Lagrangian is: $L(x,y,\nu) = f(x,y) + \nu g(x,y)$

Next, set the partial derivatives of the Lagrangian to 0:

$\frac{\partial L}{\partial x} = 2\nu x - 1 \overset{!}{=} 0$

$\frac{\partial L}{\partial y} = 4\nu y - 1 \overset{!}{=} 0$

$\frac{\partial L}{\partial \nu} = x^2 + 2 y^2 -5 \overset{!}{=} 0$

$\implies x = 1.83; y = 0.91, \nu = 0.27$

Since the problem has an equality constraint, it must be active, since the optimal point must be on the surface $g(x) = 0$.

## Exercise 2

(a) The dual problem can be formulated as:

$\max\limits_{\lambda_1,\lambda_2} g(\lambda_1,\lambda_2)$ with $g(\lambda_1,\lambda_2) = \inf \limits_{x} L(x,\lambda_1,\lambda_2) ; \quad L(x,\lambda_1,\lambda_2) = c^T x + \lambda_1^T(Ax-b) + \lambda_2^T (-x) = (c^T + \lambda_1^TA-\lambda_2^T)x - \lambda_1^Tb; \quad \lambda_1 \in \mathbb{R}^m; \lambda_2 \in \mathbb{R}^n$.

At the infimum of x, the partial derivative of the Lagrangian $\frac{\partial L}{\partial x} = (c^T + \lambda_1^TA-\lambda_2^T) = 0$. Hence the dual problem can be rewritten as:

$\max\limits_{\lambda_1, \lambda_2} (c^T + \lambda_1^TA-\lambda_2^T)x - \lambda_1^Tb $

s.t. 

$\lambda_1 \geq 0$

$\lambda_2 \geq 0$

$(c^T + \lambda_1^TA-\lambda_2^T) = 0$

which is again a linear program.


(b) The Lagrangian of the quadratic program $L(x,\lambda) = \frac{1}{2} x^TQx + c^Tx + \lambda (Ax-b)$ has the derivative $\frac{\partial L(x,\lambda)}{\partial x} = Qx + c + \lambda A$. It can be solved for $x$ only if $Q$ is positive semidefinite, i.e., invertible. The problem in Exercise 1 a (trivial) quadratic program (assuming $\mathbf{x} \equiv (x,y)^T$), since the objective function can be written in the form $x^T Q x + c^T x$, where $Q = \mathbf{0}$, which is a positive semidefinite matrix.

(c) The dual function of a quadratic problem will be of the quadratic form $ (A\lambda-b)^TQ^-1(A\lambda-c) - \lambda^Tb$. Since both the primal and the dual problem are quadratic functions, they will have one global extreme value, i.e., only one saddle point, which will be the solution to both the primal and dual problem, hence strong duality holds for quadratic programs.

## Exercise 3

The resulting hyperplane is, by definition, in canonical form iff the smallest value of $Y_i(w^TX_i+b) = 1 =: Y^{\circ}(w^TX^{\circ}+b)$, hence we can ignore all other $(X_i,Y_i)$ pairs. For simplicity, we can assume (without loss of generality) that $sign(Y^{\circ}) = 1$, hence the problem comes down to:

$min \frac{1}{2} ||w||^2 = \frac{1}{2} w^T w$

s.t.

$-w^TX^{\circ}-b-1 \leq 0$

The Lagrangian is:

$L = \frac{1}{2} w^T w + \lambda (-w^TX^{\circ}-b-1)$

$\frac{\partial L}{\partial w} = w+\lambda X^{\circ} = 0$

$\frac{\partial L}{\partial \lambda} = -w^TX^{\circ}-b-1 = 0$

$\implies w^TX^{\circ}+b = 1$, i.e., the solution will always be in canonical form.



## Exercise 4

In [2]:
import time 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.testing import assert_equal, assert_almost_equal

from sklearn import preprocessing
from sklearn.datasets import load_breast_cancer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

#  Hide warnings of LinearSVC, LogisticRegression

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

np.random.seed(42)

## a) Know the Dataset


- **Features:** 
    Variables derived from some tissue imaging method, describing the size, shape, 'coastline' and texture of the tumors
- **Labels:** 
    A binary categorization of the tumors into malignant and benign

In [3]:
dataset = load_breast_cancer()

xs = dataset.data
ys = dataset.target

print(dir(dataset))
print(dataset.feature_names)
print(dataset.target_names)

['DESCR', 'data', 'feature_names', 'filename', 'frame', 'target', 'target_names']
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
['malignant' 'benign']


Split `xs, ys` to the training set `xs_train, ys_train` (size $70\%$) and the test set `xs_test, ys_test` (size $30\%$) 

## b)/c) Optimize the Hyperparameters: Linear SVM vs Logistic Regression


In [4]:
# center and normalize

scaler = preprocessing.StandardScaler().fit(xs)
xs = scaler.transform(xs)

# split into training and test test
n_train = int(len(xs) * .7)

xs_train, xs_test = xs[:n_train], xs[n_train:]
ys_train, ys_test = ys[:n_train], ys[n_train:]

# evaluate standard SVM
svm_estimator = LinearSVC().fit(xs_train,ys_train)
print('We can do better than ', np.mean(svm_estimator.predict(xs_test) != ys_test))


We can do better than  0.03508771929824561
(398, 30)


Find good hyperparameters *without* the test set.

In [37]:
# DO NOT USE xs_test, ys_test here!
# Please make your optimization reproducable (e.g. set random_state, seed, …)
# LinearSVC and LogisticRegression
nfolds = 5
split_idx = np.linspace(0, len(ys_train),nfolds+1).astype(int)
cs = np.logspace(-5, 1, base=10, num = 100)
cverror_svc = np.zeros((nfolds, len(cs)))
cverror_lr = np.zeros((nfolds, len(cs)))
samplecounter = np.arange(len(ys_train))
for fold in range(nfolds):
    for c_idx, c in enumerate(cs):
        cv_selector = (samplecounter >= split_idx[fold]) & (samplecounter < split_idx[fold+1]) 
        train_selector  = ~cv_selector #invert cv bools to get training samples for current fold
        
        #Linear SVC
        cverror_svc[fold, c_idx] = np.mean(LinearSVC(C=c).fit(xs_train[train_selector,:],ys_train[train_selector]).predict(xs_train[cv_selector,:]) != ys_train[cv_selector])

        #same for logistic regression
        cverror_lr[fold, c_idx] = np.mean(LogisticRegression(C=c).fit(xs_train[train_selector,:],ys_train[train_selector]).predict(xs_train[cv_selector,:]) != ys_train[cv_selector])

cv_errors_svm = np.mean(cverror_svc,0)
cv_errors_lr = np.mean(cverror_lr,0)
best_params_svm, best_params_lr = {'C': cs[cv_errors_svm.argmin()]}, {'C': cs[cv_errors_lr.argmin()]}

print(f"Cross validation errors of best model: Linear SVM {cv_errors_svm.min():.4f}, Logistic Regression {cv_errors_lr.min():.4f}")


best_estimator_svm = LinearSVC(**best_params_svm).fit(xs_train,ys_train)
best_estimator_lr = LogisticRegression(**best_params_lr).fit(xs_train,ys_train)



Cross validation errors of best model: Linear SVM 0.0251, Logistic Regression 0.0276


In [36]:
# if you did not solve the cross validation, use this:
# svm_estimator = LinearSVC(C=0.0035, random_state=42).fit(xs_train, ys_train)
# lr_estimator = LogisticRegression(C=0.4037,random_state=42).fit(xs_train, ys_train)
# else, use this:
svm_estimator = best_estimator_svm
lr_estimator = best_estimator_lr

### YOUR CODE HERE
print(f"Test errors of best model: Linear SVM {np.mean(best_estimator_svm.predict(xs_test) != ys_test):.4f}, \
    Logistic Regression {np.mean(best_estimator_lr.predict(xs_test) != ys_test):.4f}")


Test errors of best model: Linear SVM 0.0117,     Logistic Regression 0.0292


What do you observe and why?

Answer: while the CV error is similar between both models (LR is a bit worse), the difference is more pronounced when it comes to the test set error. 
This could be related to the more robust algorithm in SVM, which allows it to generalize better


## d) State concerns

1. **Ethical:** 
    When tuning the classifier, we did not differentiate between sensitivity and specificity, while in the breast cancer context, we will typically want to have high sensitivity if at the expense of specificity, because missing a true malignant case would be worse than a false alarm.

2. **Technical/Statistical:**
    we have used a fairly high number of features, some of which are clearly correlated (e.g. mean perimeter, mean radius). We should have checked if the feature space dimensionality can be reduced

3. **Any:**
    We have only considered linear classifiers.